# Cell Membrane Segmentation
Segmentation of fluorescently labeled cell membranes using watershed segmentation

In [1]:
import numpy as np
from skimage import io, filters, feature, segmentation, data
from scipy import ndimage
import matplotlib.pyplot as plt

## Load and preprocess the image

In [2]:
# Load image using sample data
img = data.coins()
plt.imsave('input.png', img.astype(np.uint8), cmap='gray')

## Enhance membrane edges using Gaussian difference

In [3]:
edges = filters.gaussian(img, sigma=1) - filters.gaussian(img, sigma=2)
edges = edges / edges.max()  # normalize to 0-1
plt.imsave('edges.png', (edges * 255).astype(np.uint8), cmap='gray')

## Generate seeds for watershed using distance transform

In [4]:
distance = ndimage.distance_transform_edt(edges < 0.2)
coordinates = feature.peak_local_max(distance, min_distance=20)
local_max = np.zeros_like(distance, dtype=bool)
local_max[tuple(coordinates.T)] = True
markers = ndimage.label(local_max)[0]
plt.imsave('markers.png', markers, cmap='nipy_spectral')

## Perform watershed segmentation

In [5]:
labels = segmentation.watershed(-distance, markers, mask=edges < 0.3)
plt.imsave('labels.png', labels, cmap='nipy_spectral')
np.save('labels.npy', labels)